In [2]:
import pandas as pd
from difflib import SequenceMatcher
from datetime import datetime

# -----------------------------
# Create sample data
# -----------------------------
df_a = pd.DataFrame([
    {"customer_id": 1, "name": "John Smith", "tier": "Gold", "revenue": 1000, "updated_at": "2025-01-01", "source": "CRM"},
    {"customer_id": 2, "name": "Sara Lee", "tier": "Silver", "revenue": 500, "updated_at": "2025-01-12", "source": "CRM"}
])

df_b = pd.DataFrame([
    {"customer_id": 1, "name": "Jon Smyth", "tier": "Premium", "revenue": 1200, "updated_at": "2025-02-05", "source": "Billing"},
    {"customer_id": 2, "name": "Sara Lee", "tier": "Silver", "revenue": 500, "updated_at": "2024-12-28", "source": "Billing"}
])

# Merge datasets on key
merged = pd.merge(df_a, df_b, on="customer_id", suffixes=("_a", "_b"))

# -----------------------------
# Scoring + Comparison Logic
# -----------------------------

def similarity(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()

def compute_confidence(row):
    score = 0
    
    # Source reliability weight
    source_weight = 1 if row["source_b"] == "Billing" else 0.8
    
    # Recency weight
    recency_weight = 1 if datetime.fromisoformat(row["updated_at_b"]) > datetime.fromisoformat(row["updated_at_a"]) else 0.5

    # Consensus weight
    consensus = similarity(row["name_a"], row["name_b"])
    
    return round((source_weight*0.5 + recency_weight*0.3 + consensus*0.2), 2)

results = []

compare_fields = ["name", "tier", "revenue"]

for _, row in merged.iterrows():
    for field in compare_fields:
        val_a = row[f"{field}_a"]
        val_b = row[f"{field}_b"]
        status = "match" if val_a == val_b else ("similar" if similarity(val_a, val_b) > 0.7 else "conflict")
        
        results.append({
            "customer_id": row["customer_id"],
            "field": field,
            "value_a": val_a,
            "value_b": val_b,
            "status": status,
            "confidence_score": compute_confidence(row)
        })

summary = pd.DataFrame(results)
summary


,customer_id,field,value_a,value_b,status,confidence_score
0,1,name,John Smith,Jon Smyth,similar,0.97
1,1,tier,Gold,Premium,conflict,0.97
2,1,revenue,1000,1200,similar,0.97
3,2,name,Sara Lee,Sara Lee,match,0.85
4,2,tier,Silver,Silver,match,0.85
5,2,revenue,500,500,match,0.85
